In [2]:
pip install selenium==3.14.1

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# http://ticket.yes24.com/New/Genre/GenreMain.aspx?genre=15456&Gcode=009_202_001
import time, random, sys, os
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from twocaptcha import TwoCaptcha

In [33]:
# chromedriver 실행 권한 문제 시 터미널에서 아래 코드 실행
# xattr -d com.apple.quarantine "chromedirver 경로"
driver = webdriver.Chrome("chromedriver.exe")

# url = "http://ticket.yes24.com/New/Genre/GenreMain.aspx?genre=15456&Gcode=009_202_001"
# url = "임영웅 콘서트 예매 주소 나오면 그걸로 변경"
url = "http://ticket.yes24.com/Perf/43423"
driver.get(url)

loginA = driver.find_element_by_id("consiceLogin")

loginA.click();

time.sleep(0.5)

idInput = driver.find_element_by_id("SMemberID")
pwInput = driver.find_element_by_id("SMemberPassword")

userId = "yes24 아이디"
userPw = "yes24 비밀번호"

idInput.send_keys(userId)
pwInput.send_keys(userPw)

loginBtn = driver.find_element_by_id("btnLogin")
loginBtn.click();


time.sleep(0.5)

reserveDiv = driver.find_element_by_class_name("rn-05")

reserveATags = reserveDiv.find_elements_by_tag_name("a")

for a in reserveATags:
    if (a.text == '예매하기'):
        reserveBtn = a
        break
        
reserveBtn.click()

time.sleep(0.5)

driver.switch_to.window(driver.window_handles[1])

time.sleep(3)

# id = 2022-12-03

# 다음 달력 가야하는 경우
# calNext = driver.find_element_by_class_name("next")
# calNext.click()

targetDay = driver.find_element_by_id("2022-11-19")
targetDay.click()

time.sleep(1)

ulTime = driver.find_element_by_id("ulTime")
targetTime = ulTime.find_element_by_tag_name("li")

targetTime.click()


seatBtn = driver.find_element_by_id("btnSeatSelect")
seatBtn.click()

time.sleep(1)

def findSeats(seats, cnt):
    if not (seats == []):
        for s in seats:
            s.click()
            cnt += 1
            
            if cnt == 2:
                return cnt

# 좌석 선택
driver.switch_to.frame("ifrmSeatFrame")

isPresent = (driver.find_elements_by_name("maphall") != [])

# 미니맵이 있는 경우
if isPresent:
    
    # 미니맵 선택
    miniMap = driver.find_element_by_name("maphall")

    areas = miniMap.find_elements_by_tag_name("area")

    for area in areas:
        area.click()

        time.sleep(2)

        seatArray = driver.find_element_by_xpath('//*[@id="divSeatArray"]')

        # s9 : VIP
        # s6 : R
        # s8 : S
        seatsVIP = seatArray.find_elements_by_class_name("s9")
        seatsR = seatArray.find_elements_by_class_name("s6")
        seatsS = seatArray.find_elements_by_class_name("s8")

        cnt = 0

        resultVIP = findSeats(seatsVIP, cnt)

        if resultVIP == 2:
            break
        else:
            resultR = findSeats(seatsR, cnt)

        if resultR == 2:
            break
        else:
            resultS = findSeats(seatsS, cnt)

        if resultS == 2:
            break
            
# 미니맵이 없는 경우            
else:
    gradeBtn = driver.find_elements_by_name("btnGrade")
    
    checkCnt = False
    
    for btn in gradeBtn:
        btn.click()
        
        time.sleep(2)
        
        seatLayer = driver.find_element_by_class_name("seat_layer")
        
        lis = seatLayer.find_elements_by_tag_name("li")
        
        for li in lis:
            li.click()
            
            time.sleep(1)
            
            seatArray = driver.find_element_by_xpath('//*[@id="divSeatArray"]')

            # s9 : VIP
            # s6 : R
            # s8 : S
            seatsVIP = seatArray.find_elements_by_class_name("s9")
            
            cnt = 0
            
            if (seatsVIP != []):
                resultVIP = findSeats(seatsVIP, cnt)
                
                if resultVIP == 2:
                    checkCnt = True
                    break
            
            time.sleep(1)
            
            seatsR = seatArray.find_elements_by_class_name("s6")
            
            if (seatsR != []):
                resultR = findSeats(seatsR, cnt)
                
                if resultR == 2:
                    checkCnt = True
                    break

            time.sleep(0.3)
                    
            seatsS = seatArray.find_elements_by_class_name("s8")
            
            if (seatsS != []):
                resultS = findSeats(seatsS, cnt)
                
                if resultS == 2:
                    checkCnt = True
                    break           
        if checkCnt:
            break


    
# 좌석 선택 이후
bookingBtn = driver.find_element_by_class_name("booking")
bookingBtn.click()

time.sleep(2)

# 선택 이후 부분
driver.switch_to.default_content()

time.sleep(2)

nextBtn = driver.find_element_by_id("StepCtrlBtn03")

_next = nextBtn.find_element_by_xpath('//*[@id="StepCtrlBtn03"]/a[2]/img')
_next.click()

time.sleep(2)

nextBtn2 = driver.find_element_by_id("StepCtrlBtn04")
_next2 = nextBtn2.find_element_by_xpath('//*[@id="StepCtrlBtn04"]/a[2]/img')
_next2.click()

time.sleep(3)

radio = driver.find_element_by_xpath('//*[@id="rdoPays37433"]')                                
radio.click()

bank = Select(driver.find_element_by_id("selBank"))
bank.select_by_index(1)

agree1 = driver.find_element_by_id("cbxCancelFeeAgree")
agree2 = driver.find_element_by_id("chkinfoAgree")

agree1.click()
agree2.click()                

In [30]:
# 네이버 로그인 => 사용하면 자동 로그인 방지뜸
driver.switch_to.window(driver.window_handles[2])

naverId = "네이버 ID"
naverPw = "네이버 비밀번호"

naverIdInput = driver.find_element_by_id("id")
naverPwInput = driver.find_element_by_id("pw")

naverIdInput.send_keys(naverId)
naverPwInput.send_keys(naverPw)

naverBtn = driver.find_element_by_id("log.login")
naverBtn.click()